### Importing the required libraries and dataset

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

In [2]:
data=pd.read_excel(r"D:\Elahe\MSF\Thesis\Datasets\sample.xlsx")

#### Defining a division function

In [3]:
def Division (numerator,denominator):
    result=[]
    for i in range (len(numerator)):
        if numerator[i]!="." and denominator[i]!=0 and denominator[i]!=".":
            result.append(numerator[i]/denominator[i])
        else:
            result.append(".")
    return result

#### Calculating daily return and shorting daily return for bull, bear and index

In [4]:
def daily_return (price, ticker):
    daily_return = ["."]
    for i in range (1, len(price)):
        if ticker[i]==ticker[i-1]:
            daily_return.append((price[i]-price[i-1])/price[i-1])
        else:
            daily_return.append(".")
    return daily_return

In [5]:
def short_daily_return (daily_return):
    short_return=[]
    for i in range (len(daily_return)):
        if daily_return[i] != ".":
            short_return.append((daily_return[i]*-1))
        else:
            short_return.append(".")
    return short_return

#### Calculating cumulative return

In [6]:
def Cumulative_Long_Return (daily_return, period):
    Cumulative_Return=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            Cumulative_Return.append(".")
        elif daily_return[i-1]==".":
            Cumulative_Return.append(daily_return[i]+1)
        elif period[i-1]==period[i]:
            Cumulative_Return.append(Cumulative_Return[i-1]*(daily_return[i]+1))
        else:
            Cumulative_Return.append(daily_return[i]+1)
    for j in range (len(Cumulative_Return)):
        if Cumulative_Return[j]!=".":
            Cumulative_Return[j]=Cumulative_Return[j]-1
    return Cumulative_Return

In [7]:
def Cumulative_Short_Return (daily_return, period):
    Cumulative_Return=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            Cumulative_Return.append(".")
        elif daily_return[i-1]==".":
            Cumulative_Return.append(daily_return[i]-1)
        elif period[i-1]==period[i]:
            Cumulative_Return.append(Cumulative_Return[i-1]*(daily_return[i]-1))
        else:
            Cumulative_Return.append(daily_return[i]-1)
    for j in range (len(Cumulative_Return)):
        if Cumulative_Return[j]!=".":
            Cumulative_Return[j]=1-abs(Cumulative_Return[j])
    return Cumulative_Return

#### Calculating sharpe ratio

In [8]:
def Sharpe_Ratio (daily_return, Rf, period):
    Partial_Data=[]
    Sharpe_Ratio=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            Sharpe_Ratio.append(".")
        elif i == len(daily_return)-1:
            Partial_Data.append(daily_return[i]-Rf[i])
            Sharpe_Ratio.append(np.mean(Partial_Data)/np.std(Partial_Data))
        elif period[i]==period[i+1]:
            Partial_Data.append(daily_return[i]-Rf[i])
            Sharpe_Ratio.append(".")
        else:
            Partial_Data.append(daily_return[i]-Rf[i])
            Sharpe_Ratio.append(np.mean(Partial_Data)/np.std(Partial_Data))
            Partial_Data=[]
    return Sharpe_Ratio

#### Calculating Sortino ratio

In [9]:
def Sortino_Ratio (daily_return, Rf, period):
    Partial_Data=[]
    negative_return=[]
    Sortino_Ratio=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            Sortino_Ratio.append(".")
        elif i == len(daily_return)-1:
            Partial_Data.append(daily_return[i]-Rf[i])
            if daily_return[i]-Rf[i]<0:
                negative_return.append((daily_return[i]-Rf[i])**2)
            Sortino_Ratio.append(np.mean(Partial_Data)/((np.sum(negative_return)/len(negative_return))**(1/2)))
        elif period[i]==period[i+1]:
            Partial_Data.append(daily_return[i]-Rf[i])
            Sortino_Ratio.append(".")
            if daily_return[i]-Rf[i]<0:
                negative_return.append((daily_return[i]-Rf[i])**2)
        else:
            Partial_Data.append(daily_return[i]-Rf[i])
            if daily_return[i]-Rf[i]<0:
                negative_return.append((daily_return[i]-Rf[i])**2)
            Sortino_Ratio.append(np.mean(Partial_Data)/((np.sum(negative_return)/len(negative_return))**(1/2)))
            Partial_Data=[]
            negative_return=[]
    return Sortino_Ratio

#### Calculating analytical VaR

In [10]:
def Analytical_VaR (daily_return, period):
    Partial_Data=[]
    Analytical_VaR=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            Analytical_VaR.append(".")
        elif i == len(daily_return)-1:
            Partial_Data.append(daily_return[i])
            Analytical_VaR.append(np.mean(Partial_Data)-(1.65*np.std(Partial_Data)))
        elif period[i]==period[i+1]:
            Partial_Data.append(daily_return[i])
            Analytical_VaR.append(".")
        else:
            Partial_Data.append(daily_return[i])
            Analytical_VaR.append(np.mean(Partial_Data)-(1.65*np.std(Partial_Data)))
            Partial_Data=[]
    return Analytical_VaR

#### Calculating VaR

In [11]:
def VaR (daily_return, period):
    Partial_Data=[]
    VaR=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            VaR.append(".")
        elif i == len(daily_return)-1:
            Partial_Data.append(daily_return[i])
            Partial_Data.sort()
            VaR.append(Partial_Data[round(0.05*len(Partial_Data))-1])
        elif period[i]==period[i+1]:
            Partial_Data.append(daily_return[i])
            VaR.append(".")
        else:
            Partial_Data.append(daily_return[i])
            Partial_Data.sort()
            VaR.append(Partial_Data[round(0.05*len(Partial_Data))-1])
            Partial_Data=[]
    return VaR

#### Calculating expected VaR

In [12]:
def Expected_VaR (daily_return, period):
    Partial_Data=[]
    Expected_VaR=[]
    Sum_Left_Tail=0
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            Expected_VaR.append(".")
        elif i == len(daily_return)-1:
            Partial_Data.append(daily_return[i])
            Partial_Data.sort()
            for j in range(round(0.05*len(Partial_Data))):
                Sum_Left_Tail+=Partial_Data[j]
            Expected_VaR.append(Sum_Left_Tail/round(0.05*len(Partial_Data)))
        elif period[i]==period[i+1]:
            Partial_Data.append(daily_return[i])
            Expected_VaR.append(".")
        else:
            Partial_Data.append(daily_return[i])
            Partial_Data.sort()
            for j in range(round(0.05*len(Partial_Data))):
                Sum_Left_Tail+=Partial_Data[j]
            Expected_VaR.append(Sum_Left_Tail/round(0.05*len(Partial_Data)))
            Partial_Data=[]
            Sum_Left_Tail=0
    return Expected_VaR

#### Creating portfolios and finding the best strategy

In [13]:
def Portfolio (Bull_Return, Bear_Return, Bull_Weight, Bear_Weight):
    portfolio_return=[]
    for i in range (len(Bull_Return)):
        if Bull_Return[i] != ".":
            portfolio_return.append((Bull_Return[i]*Bull_Weight)+(Bear_Return[i]*Bear_Weight))
        else:
            portfolio_return.append(".")
    return portfolio_return

In [14]:
def Best_Strategy (P1,P2,P3,P4,P5,P6,P7):
    Highest_Return=[]
    Best_Strategy=[]
    for i in range (len(P1)):
        if P1[i]!=".":
            Highest_Return.append(max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]))
            if max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P1[i]:
                Best_Strategy.append("1:0")
            elif max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P2[i]:
                Best_Strategy.append("0.75:0.25")
            elif max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P3[i]:
                Best_Strategy.append("0.67:0.33")
            elif max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P4[i]:
                Best_Strategy.append("0.5:0.5")
            elif max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P5[i]:
                Best_Strategy.append("0.33:0.67")
            elif max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P6[i]:
                Best_Strategy.append("0.25:0.75")
            elif max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P7[i]:
                Best_Strategy.append("0:1")
        else:
            Highest_Return.append(".")
            Best_Strategy.append(".")
    return Highest_Return, Best_Strategy

### Removing empty observations

In [15]:
def Remove_Character (dataset, list_name, Character):
    removal_rows=[]
    for i in range (len(list_name)):
        if list_name[i]==Character:
            removal_rows.append(i)
    dataset=dataset.drop(removal_rows)
    dataset=dataset.reset_index()
    return dataset

### Defining a main function to call all functions at the same time

In [16]:
def Main_Function (data, period, investment):
    
    # Calculating daily returns
    data["Bull Return Long"]=daily_return(data["Bull Price"],data["Code"])
    data["Bear Return Long"]=daily_return(data["Bear Price"],data["Code"])
    data["Index Return"]=daily_return(data["Index Price"],data["Code"])
    
    # Calculating short daily returns
    data["Bull Return Short"]=short_daily_return(data["Bull Return Long"])
    data["Bear Return Short"]=short_daily_return(data["Bear Return Long"])
    
    # Calculating cumulative return
    if investment == "Long":
        data["Index CuRe"]=Cumulative_Long_Return(data["Index Return"],data[period])
        data["Bull CuRe "+investment]=Cumulative_Long_Return(data["Bull Return "+investment],data[period])
        data["Bear CuRe "+investment]=Cumulative_Long_Return(data["Bear Return "+investment],data[period])
    else:        
        data["Bull CuRe "+investment]=Cumulative_Short_Return(data["Bull Return "+investment],data[period])
        data["Bear CuRe "+investment]=Cumulative_Short_Return(data["Bear Return "+investment],data[period])

    # Calculating Sharpe ratio
    data["Index Sharpe"]=Sharpe_Ratio(data["Index Return"],data["Rf"],data[period])
    data["Bull Sharpe Ratio "+investment]=Sharpe_Ratio(data["Bull Return "+investment],data["Rf"],data[period])
    data["Bear Sharpe Ratio "+investment]=Sharpe_Ratio(data["Bear Return "+investment],data["Rf"],data[period])
    
    # Calculating Sortino ratio
    data["Index Sortino"]=Sortino_Ratio(data["Index Return"],data["Rf"],data[period])
    data["Bull Sortino Ratio "+investment]=Sortino_Ratio(data["Bull Return "+investment],data["Rf"],data[period])
    data["Bear Sortino Ratio "+investment]=Sortino_Ratio(data["Bear Return "+investment],data["Rf"],data[period])
    
    # Calculating Analytical VaR
    data["Index Analytical VaR"]=Analytical_VaR(data["Index Return"],data[period])
    data["Bull Analytical VaR "+investment]=Analytical_VaR(data["Bull Return "+investment],data[period])
    data["Bear Analytical VaR "+investment]=Analytical_VaR(data["Bear Return "+investment],data[period])
    
    # Calculating VaR
    data["Index VaR"]=VaR(data["Index Return"],data[period])
    data["Bull VaR "+investment]=VaR(data["Bull Return "+investment],data[period])
    data["Bear VaR "+investment]=VaR(data["Bear Return "+investment],data[period])
    
    # Calculating Expected VaR
    data["Index EVaR"]=Expected_VaR(data["Index Return"],data[period])
    data["Bull Expected VaR "+investment]=Expected_VaR(data["Bull Return "+investment],data[period])
    data["Bear Expected VaR "+investment]=Expected_VaR(data["Bear Return "+investment],data[period])
    
    # Calculating the return of each strategy using cumulative return results
    data["CuRe-Portfolio 1:0 "+investment]=Portfolio(data["Bull CuRe "+investment],data["Bear CuRe "+investment],1,0)
    data["CuRe-Portfolio 0.75:0.25 "+investment]=Portfolio(data["Bull CuRe "+investment],data["Bear CuRe "+investment],0.75,0.25)
    data["CuRe-Portfolio 0.67:0.33 "+investment]=Portfolio(data["Bull CuRe "+investment],data["Bear CuRe "+investment],0.67,0.33)
    data["CuRe-Portfolio 0.5:0.5 "+investment]=Portfolio(data["Bull CuRe "+investment],data["Bear CuRe "+investment],0.5,0.5)
    data["CuRe-Portfolio 0.33:0.67 "+investment]=Portfolio(data["Bull CuRe "+investment],data["Bear CuRe "+investment],0.33,0.67)
    data["CuRe-Portfolio 0.25:0.75 "+investment]=Portfolio(data["Bull CuRe "+investment],data["Bear CuRe "+investment],0.25,0.75)
    data["CuRe-Portfolio 0:1 "+investment]=Portfolio(data["Bull CuRe "+investment],data["Bear CuRe "+investment],0,1)
    
    # Finding the best strategy using cumulative return results
    data["CuRe-Portfolio Highest_Return "+investment], data["CuRe-Portfolio Best_Strategy "+investment] = Best_Strategy(
    data["CuRe-Portfolio 1:0 "+investment],data["CuRe-Portfolio 0.75:0.25 "+investment], data["CuRe-Portfolio 0.67:0.33 "+investment], 
    data["CuRe-Portfolio 0.5:0.5 "+investment],data["CuRe-Portfolio 0.33:0.67 "+investment],data["CuRe-Portfolio 0.25:0.75 "+investment]
    , data["CuRe-Portfolio 0:1 "+investment])
    
    # Calculating the return of each strategy using using Sharpe ratio results
    data["Sharpe-Portfolio 1:0 "+investment]=Portfolio(data["Bull Sharpe Ratio "+investment],data["Bear Sharpe Ratio "+investment],1,0)
    data["Sharpe-Portfolio 0.75:0.25 "+investment]=Portfolio(data["Bull Sharpe Ratio "+investment],data["Bear Sharpe Ratio "+investment],0.75,0.25)
    data["Sharpe-Portfolio 0.67:0.33 "+investment]=Portfolio(data["Bull Sharpe Ratio "+investment],data["Bear Sharpe Ratio "+investment],0.67,0.33)
    data["Sharpe-Portfolio 0.5:0.5 "+investment]=Portfolio(data["Bull Sharpe Ratio "+investment],data["Bear Sharpe Ratio "+investment],0.5,0.5)
    data["Sharpe-Portfolio 0.33:0.67 "+investment]=Portfolio(data["Bull Sharpe Ratio "+investment],data["Bear Sharpe Ratio "+investment],0.33,0.67)
    data["Sharpe-Portfolio 0.25:0.75 "+investment]=Portfolio(data["Bull Sharpe Ratio "+investment],data["Bear Sharpe Ratio "+investment],0.25,0.75)
    data["Sharpe-Portfolio 0:1 "+investment]=Portfolio(data["Bull Sharpe Ratio "+investment],data["Bear Sharpe Ratio "+investment],0,1)
    
    # Finding the best strategy using Sharpe ratio results
    data["Sharpe-Portfolio Highest_Return "+investment], data["Sharpe-Portfolio Best_Strategy "+investment] = Best_Strategy(
    data["Sharpe-Portfolio 1:0 "+investment],data["Sharpe-Portfolio 0.75:0.25 "+investment], 
    data["Sharpe-Portfolio 0.67:0.33 "+investment], data["Sharpe-Portfolio 0.5:0.5 "+investment],
    data["Sharpe-Portfolio 0.33:0.67 "+investment],data["Sharpe-Portfolio 0.25:0.75 "+investment], 
    data["Sharpe-Portfolio 0:1 "+investment])
    
    # Calculating the return of each strategy using using Sortino ratio results
    data["Sortino-Portfolio 1:0 "+investment]=Portfolio(data["Bull Sortino Ratio "+investment],data["Bear Sortino Ratio "+investment],1,0)
    data["Sortino-Portfolio 0.75:0.25 "+investment]=Portfolio(data["Bull Sortino Ratio "+investment],data["Bear Sortino Ratio "+investment],0.75,0.25)
    data["Sortino-Portfolio 0.67:0.33 "+investment]=Portfolio(data["Bull Sortino Ratio "+investment],data["Bear Sortino Ratio "+investment],0.67,0.33)
    data["Sortino-Portfolio 0.5:0.5 "+investment]=Portfolio(data["Bull Sortino Ratio "+investment],data["Bear Sortino Ratio "+investment],0.5,0.5)
    data["Sortino-Portfolio 0.33:0.67 "+investment]=Portfolio(data["Bull Sortino Ratio "+investment],data["Bear Sortino Ratio "+investment],0.33,0.67)
    data["Sortino-Portfolio 0.25:0.75 "+investment]=Portfolio(data["Bull Sortino Ratio "+investment],data["Bear Sortino Ratio "+investment],0.25,0.75)
    data["Sortino-Portfolio 0:1 "+investment]=Portfolio(data["Bull Sortino Ratio "+investment],data["Bear Sortino Ratio "+investment],0,1)
    
    # Finding the best strategy using Sortino ratio results
    data["Sortino-Portfolio Highest_Return "+investment], data["Sortino-Portfolio Best_Strategy "+investment] = Best_Strategy(
    data["Sortino-Portfolio 1:0 "+investment],data["Sortino-Portfolio 0.75:0.25 "+investment], 
    data["Sortino-Portfolio 0.67:0.33 "+investment], data["Sortino-Portfolio 0.5:0.5 "+investment],
    data["Sortino-Portfolio 0.33:0.67 "+investment],data["Sortino-Portfolio 0.25:0.75 "+investment], 
    data["Sortino-Portfolio 0:1 "+investment])
    
    # Calculating the return of each strategy using expected VaR results
    data["EVaR-Portfolio 1:0 "+investment]=Portfolio(data["Bull Expected VaR "+investment],data["Bear Expected VaR "+investment],1,0)
    data["EVaR-Portfolio 0.75:0.25 "+investment]=Portfolio(data["Bull Expected VaR "+investment],data["Bear Expected VaR "+investment],0.75,0.25)
    data["EVaR-Portfolio 0.67:0.33 "+investment]=Portfolio(data["Bull Expected VaR "+investment],data["Bear Expected VaR "+investment],0.67,0.33)
    data["EVaR-Portfolio 0.5:0.5 "+investment]=Portfolio(data["Bull Expected VaR "+investment],data["Bear Expected VaR "+investment],0.5,0.5)
    data["EVaR-Portfolio 0.33:0.67 "+investment]=Portfolio(data["Bull Expected VaR "+investment],data["Bear Expected VaR "+investment],0.33,0.67)
    data["EVaR-Portfolio 0.25:0.75 "+investment]=Portfolio(data["Bull Expected VaR "+investment],data["Bear Expected VaR "+investment],0.25,0.75)
    data["EVaR-Portfolio 0:1 "+investment]=Portfolio(data["Bull Expected VaR "+investment],data["Bear Expected VaR "+investment],0,1)
    
    # Finding the best strategy using expected VaR results
    data["EVaR-Portfolio Highest_Return "+investment], data["EVaR-Portfolio Best_Strategy "+investment] = Best_Strategy(
    data["EVaR-Portfolio 1:0 "+investment],data["EVaR-Portfolio 0.75:0.25 "+investment], 
    data["EVaR-Portfolio 0.67:0.33 "+investment], data["EVaR-Portfolio 0.5:0.5 "+investment],
    data["EVaR-Portfolio 0.33:0.67 "+investment],data["EVaR-Portfolio 0.25:0.75 "+investment], 
    data["EVaR-Portfolio 0:1 "+investment])

### Creating two datasets: quarterly analysis and annual analysis

In [17]:
data_quarter=data.copy()
data_yearly=data.copy()

### Calculating all portfolios using longing and shorting

In [18]:
Main_Function(data_quarter, "Quarter", "Long")
Main_Function(data_yearly, "Year", "Long")

In [19]:
Main_Function(data_quarter, "Quarter", "Short")
Main_Function(data_yearly, "Year", "Short")

#### Calculating the trend of the prices

In [20]:
def Trend (daily_return, period, starting, ending):
    Partial_Data=[]
    Trend=[]
    Trend_Dummy=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            Trend.append(".")
        elif i == len(daily_return)-1:
            Partial_Data.append(daily_return[i])
            Last_Days_Return_Starting = Partial_Data[len(Partial_Data)-starting:]
            Last_Days_Return_Ending = Partial_Data[len(Partial_Data)-ending:]
            Trend.append(np.mean(Last_Days_Return_Ending)-np.mean(Last_Days_Return_Starting))
        elif period[i]==period[i+1]:
            Partial_Data.append(daily_return[i])
            Trend.append(".")
        else:
            Partial_Data.append(daily_return[i])
            Last_Days_Return_Starting = Partial_Data[len(Partial_Data)-starting:]
            Last_Days_Return_Ending = Partial_Data[len(Partial_Data)-ending:]
            Trend.append(np.mean(Last_Days_Return_Ending)-np.mean(Last_Days_Return_Starting))
            Partial_Data=[]
    for i in range (len(Trend)):
        if Trend[i]>=0:
            Trend_Dummy[i]=1
        else:
            Trend_Dummy[i]=0
    return Trend_Dummy

In [21]:
data_quarter["Bull Trend"]=Trend(data_quarter["Bull Return Long"],data["Quarter"],12,33)
data_quarter["Bear Trend"]=Trend(data_quarter["Bear Return Long"],data["Quarter"],12,33)
data_quarter["Index Trend"]=Trend(data_quarter["Index Return"],data["Quarter"],12,33)

In [22]:
data_yearly["Bull Trend"]=Trend(data_yearly["Bull Return Long"],data["Year"],43,152)
data_yearly["Bear Trend"]=Trend(data_yearly["Bear Return Long"],data["Year"],43,152)
data_yearly["Index Trend"]=Trend(data_yearly["Index Return"],data["Year"],43,152)

#### Calculating the volatility

In [23]:
def Volatility (daily_return, period):
    Partial_Data=[]
    Volatility=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            Volatility.append(".")
        elif i == len(daily_return)-1:
            Partial_Data.append(daily_return[i])
            Volatility.append(np.std(Partial_Data))
        elif period[i]==period[i+1]:
            Partial_Data.append(daily_return[i])
            Volatility.append(".")
        else:
            Partial_Data.append(daily_return[i])
            Volatility.append(np.std(Partial_Data))
            Partial_Data=[]
    return Volatility

In [24]:
data_quarter["Bull Volatility"]=Volatility(data_quarter["Bull Return Long"],data["Quarter"])
data_quarter["Bear Volatility"]=Volatility(data_quarter["Bear Return Long"],data["Quarter"])
data_quarter["Index Volatility"]=Volatility(data_quarter["Index Return"],data["Quarter"])

In [25]:
data_yearly["Bull Volatility"]=Volatility(data_yearly["Bull Return Long"],data["Year"])
data_yearly["Bear Volatility"]=Volatility(data_yearly["Bear Return Long"],data["Year"])
data_yearly["Index Volatility"]=Volatility(data_yearly["Index Return"],data["Year"])

#### Calculating the beta of each LETF

In [26]:
def Beta (daily_return_stock, daily_return_index, period):
    Partial_Data_stock=[]
    Partial_Data_index=[]
    Beta=[]
    for i in range (len(daily_return_stock)):
        if daily_return_stock[i]==".":
            Beta.append(".")
        elif i == len(daily_return_stock)-1:
            Partial_Data_stock.append(daily_return_stock[i])
            Partial_Data_index.append(daily_return_index[i])
            Corr=st.pearsonr(Partial_Data_stock,Partial_Data_index)
            Beta.append(Corr[0])
        elif period[i]==period[i+1]:
            Partial_Data_stock.append(daily_return_stock[i])
            Partial_Data_index.append(daily_return_index[i])
            Beta.append(".")
        else:
            Partial_Data_stock.append(daily_return_stock[i])
            Partial_Data_index.append(daily_return_index[i])
            Corr=st.pearsonr(Partial_Data_stock,Partial_Data_index)
            Beta.append(Corr[0])
            Partial_Data_stock=[]
            Partial_Data_index=[]
    return Beta

In [27]:
data_quarter["Bull Beta Price"]=Beta(data_quarter["Bull Price"],data_quarter["Index Price"], data_quarter["Quarter"])
data_quarter["Bear Beta Price"]=Beta(data_quarter["Bear Price"],data_quarter["Index Price"], data_quarter["Quarter"])
data_quarter["Bull Beta Return"]=Beta(data_quarter["Bull Return Long"],data_quarter["Index Return"], data_quarter["Quarter"])
data_quarter["Bear Beta Return"]=Beta(data_quarter["Bear Return Long"],data_quarter["Index Return"], data_quarter["Quarter"])

In [28]:
data_yearly["Bull Beta Price"]=Beta(data_yearly["Bull Price"],data_yearly["Index Price"], data_yearly["Year"])
data_yearly["Bear Beta Price"]=Beta(data_yearly["Bear Price"],data_yearly["Index Price"], data_yearly["Year"])
data_yearly["Bull Beta Return"]=Beta(data_yearly["Bull Return Long"],data_yearly["Index Return"], data_yearly["Year"])
data_yearly["Bear Beta Return"]=Beta(data_yearly["Bear Return Long"],data_yearly["Index Return"], data_yearly["Year"])

#### Transforming daily data into quarterly and annual data

In [29]:
def periodical_average (daily_return, period):
    Partial_Data=[]
    periodical_average=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            periodical_average.append(".")
        elif i == len(daily_return)-1:
            Partial_Data.append(daily_return[i])
            periodical_average.append(np.mean(Partial_Data))
        elif period[i]==period[i+1]:
            Partial_Data.append(daily_return[i])
            periodical_average.append(".")
        else:
            Partial_Data.append(daily_return[i])
            periodical_average.append(np.mean(Partial_Data))
            Partial_Data=[]
    return periodical_average

###### Transforming price

In [30]:
data_quarter["Bull Ave Price"]=periodical_average(data_quarter["Bull Price"],data_quarter["Quarter"])
data_quarter["Bear Ave Price"]=periodical_average(data_quarter["Bear Price"],data_quarter["Quarter"])
data_quarter["Index Ave Price"]=periodical_average(data_quarter["Index Price"],data_quarter["Quarter"])

In [31]:
data_yearly["Bull Ave Price"]=periodical_average(data_yearly["Bull Price"],data_yearly["Year"])
data_yearly["Bear Ave Price"]=periodical_average(data_yearly["Bear Price"],data_yearly["Year"])
data_yearly["Index Ave Price"]=periodical_average(data_yearly["Index Price"],data_yearly["Year"])

###### Transforming return

In [32]:
data_quarter["Bull Ave Return"]=periodical_average(data_quarter["Bull Return Long"],data_quarter["Quarter"])
data_quarter["Bear Ave Return"]=periodical_average(data_quarter["Bear Return Long"],data_quarter["Quarter"])
data_quarter["Index Ave Return"]=periodical_average(data_quarter["Index Return"],data_quarter["Quarter"])

In [33]:
data_yearly["Bull Ave Return"]=periodical_average(data_yearly["Bull Return Long"],data_yearly["Year"])
data_yearly["Bear Ave Return"]=periodical_average(data_yearly["Bear Return Long"],data_yearly["Year"])
data_yearly["Index Ave Return"]=periodical_average(data_yearly["Index Return"],data_yearly["Year"])

###### Transforming volume

In [34]:
data_quarter["Bull Ave Volume"]=periodical_average(data_quarter["Bull Volume"],data_quarter["Quarter"])
data_quarter["Bear Ave Volume"]=periodical_average(data_quarter["Bear Volume"],data_quarter["Quarter"])
data_quarter["Index Ave Volume"]=periodical_average(data_quarter["Index Volume"],data_quarter["Quarter"])

In [35]:
data_yearly["Bull Ave Volume"]=periodical_average(data_yearly["Bull Volume"],data_yearly["Year"])
data_yearly["Bear Ave Volume"]=periodical_average(data_yearly["Bear Volume"],data_yearly["Year"])
data_yearly["Index Ave Volume"]=periodical_average(data_yearly["Index Volume"],data_yearly["Year"])

###### Transforming NAV

In [36]:
data_quarter["Bull Ave NAV"]=periodical_average(data_quarter["Bull NAV"],data_quarter["Quarter"])
data_quarter["Bear Ave NAV"]=periodical_average(data_quarter["Bear NAV"],data_quarter["Quarter"])

In [37]:
data_yearly["Bull Ave NAV"]=periodical_average(data_yearly["Bull NAV"],data_yearly["Year"])
data_yearly["Bear Ave NAV"]=periodical_average(data_yearly["Bear NAV"],data_yearly["Year"])

###### Transforming market capitalization

In [38]:
data_quarter["Bull Ave MCap"]=periodical_average(data_quarter["Bull Market Cap"],data_quarter["Quarter"])
data_quarter["Bear Ave MCap"]=periodical_average(data_quarter["Bear Market Cap"],data_quarter["Quarter"])
data_quarter["Index Ave MCap"]=periodical_average(data_quarter["Index Market Cap"],data_quarter["Quarter"])

In [39]:
data_yearly["Bull Ave MCap"]=periodical_average(data_yearly["Bull Market Cap"],data_yearly["Year"])
data_yearly["Bear Ave MCap"]=periodical_average(data_yearly["Bear Market Cap"],data_yearly["Year"])
data_yearly["Index Ave MCap"]=periodical_average(data_yearly["Index Market Cap"],data_yearly["Year"])

###### Transforming bid ask spread

In [40]:
data_quarter["Bull Ave Spread"]=periodical_average(data_quarter["Bull Bid Ask Spread"],data_quarter["Quarter"])
data_quarter["Bear Ave Spread"]=periodical_average(data_quarter["Bear Bid Ask Spread"],data_quarter["Quarter"])

In [41]:
data_yearly["Bull Ave Spread"]=periodical_average(data_yearly["Bull Bid Ask Spread"],data_yearly["Year"])
data_yearly["Bear Ave Spread"]=periodical_average(data_yearly["Bear Bid Ask Spread"],data_yearly["Year"])

###### Transforming turnover

In [42]:
data_quarter["Bull Ave Turnover"]=periodical_average(data_quarter["Bull Turnover"],data_quarter["Quarter"])
data_quarter["Bear Ave Turnover"]=periodical_average(data_quarter["Bear Turnover"],data_quarter["Quarter"])

In [43]:
data_yearly["Bull Ave Turnover"]=periodical_average(data_yearly["Bull Turnover"],data_yearly["Year"])
data_yearly["Bear Ave Turnover"]=periodical_average(data_yearly["Bear Turnover"],data_yearly["Year"])

#### Calculating price to NAV ratio

In [44]:
data_quarter["Bull Price To Nav"]=Division(data_quarter["Bull Price"],data_quarter["Bull NAV"])
data_quarter["Bear Price To Nav"]=Division(data_quarter["Bear Price"],data_quarter["Bear NAV"])

In [45]:
data_yearly["Bull Price To Nav"]=Division(data_yearly["Bull Price"],data_yearly["Bull NAV"])
data_yearly["Bear Price To Nav"]=Division(data_yearly["Bear Price"],data_yearly["Bear NAV"])

#### Transforming price to NAV

In [46]:
data_quarter["Bull Ave PTNAV"]=periodical_average(data_quarter["Bull Price To Nav"],data_quarter["Quarter"])
data_quarter["Bear Ave PTNAV"]=periodical_average(data_quarter["Bear Price To Nav"],data_quarter["Quarter"])

In [47]:
data_yearly["Bull Ave PTNAV"]=periodical_average(data_yearly["Bull Price To Nav"],data_yearly["Year"])
data_yearly["Bear Ave PTNAV"]=periodical_average(data_yearly["Bear Price To Nav"],data_yearly["Year"])

#### Defining market situation

In [48]:
def Market_Situation (dataset, daily_return, period, subperiod):
    dataset=dataset.assign(subperiod_average=periodical_average(daily_return, subperiod))
    dataset=Remove_Character(dataset, dataset["subperiod_average"],".")
    Market=[]
    for i in range (len(dataset["subperiod_average"])):
        Market.append(".")
    for i in range (len(dataset["subperiod_average"])):
        if period == "Quarter":
            if i == 0 or dataset[period][i] != dataset[period][i-1]:
                if dataset["subperiod_average"][i]<0 and dataset["subperiod_average"][i+1]<0 and dataset["subperiod_average"][i+2]<0:
                    Market[i+2]="Bear"
                elif dataset["subperiod_average"][i]>0 and dataset["subperiod_average"][i+1]>0 and dataset["subperiod_average"][i+2]>0:
                    Market[i+2]="Bull"
                else:
                    Market[i+2]="Inconsistant"
        if period == "Year":
            if i == 0 or dataset[period][i] != dataset[period][i-1]:
                if dataset["subperiod_average"][i]<0 and dataset["subperiod_average"][i+1]<0 and dataset["subperiod_average"][i+2]<0 and dataset["subperiod_average"][i+3]<0:
                    Market[i+3]="Bear"
                elif dataset["subperiod_average"][i]>0 and dataset["subperiod_average"][i+1]>0 and dataset["subperiod_average"][i+2]>0 and dataset["subperiod_average"][i+3]>0:
                    Market[i+3]="Bull"
                else:
                    Market[i+3]="Inconsistant"
    dataset=dataset.assign(Market_situation=Market)
    return dataset

In [49]:
data_quarter=Market_Situation(data_quarter,data_quarter["Index Return"],"Quarter",data_quarter["Month"])
data_yearly=Market_Situation(data_yearly,data_yearly["Index Return"],"Year",data_yearly["Quarter"])

### Calculating a dummy variable for dependent variable (First layer)

In [50]:
def First_Dependent_Variable (first_benchmark,second_benchmark,first_index,second_index):
    dependent_variable=[]
    for i in range (len(first_benchmark)):
        if first_benchmark[i] > first_index[i] and second_benchmark[i] > second_index[i]:
            dependent_variable.append(1)
        elif first_benchmark[i] <= first_index[i] or second_benchmark[i] <= second_index[i]:
            dependent_variable.append(0)
        else:
            dependent_variable.append(".")
    return dependent_variable

In [51]:
data_quarter["Long Dummy Y Stock/LETF"]=First_Dependent_Variable(data_quarter["Sharpe-Portfolio Highest_Return Long"],
                                                    data_quarter["Sortino-Portfolio Highest_Return Long"], 
                                                    data_quarter["Index Sharpe"], data_quarter["Index Sortino"])
data_yearly["Long Dummy Y Stock/LETF"]=First_Dependent_Variable(data_yearly["Sharpe-Portfolio Highest_Return Long"],
                                                    data_yearly["Sortino-Portfolio Highest_Return Long"],
                                                    data_yearly["Index Sharpe"], data_yearly["Index Sortino"])

In [52]:
data_quarter["Short Dummy Y Stock/LETF"]=First_Dependent_Variable(data_quarter["Sharpe-Portfolio Highest_Return Short"],
                                                    data_quarter["Sortino-Portfolio Highest_Return Short"],
                                                    data_quarter["Index Sharpe"], data_quarter["Index Sortino"])
data_yearly["Short Dummy Y Stock/LETF"]=First_Dependent_Variable(data_yearly["Sharpe-Portfolio Highest_Return Short"],
                                                    data_yearly["Sortino-Portfolio Highest_Return Short"],
                                                    data_yearly["Index Sharpe"], data_yearly["Index Sortino"])

### Calculating a dummy variable for dependent variable (second layer)

In [53]:
def Second_Dependent_Variable (first_benchmark,second_benchmark,first_strategy,second_strategy):
    dependent_variable=[]
    for i in range (len(first_benchmark)):
        if first_benchmark[i] == second_benchmark[i] == first_strategy:
            dependent_variable.append(0)
        elif first_benchmark[i] == second_benchmark[i] == second_strategy:
            dependent_variable.append(1)
        else:
            dependent_variable.append(".")
    return dependent_variable

In [54]:
data_quarter["Long Dummy Y Bull/Bear"]=Second_Dependent_Variable(data_quarter["Sharpe-Portfolio Best_Strategy Long"],
                                                      data_quarter["Sortino-Portfolio Best_Strategy Long"],"1:0","0:1")
data_yearly["Long Dummy Y Bull/Bear"]=Second_Dependent_Variable(data_yearly["Sharpe-Portfolio Best_Strategy Long"],
                                                      data_yearly["Sortino-Portfolio Best_Strategy Long"],"1:0","0:1")

In [55]:
data_quarter["Short Dummy Y Bull/Bear"]=Second_Dependent_Variable(data_quarter["Sharpe-Portfolio Best_Strategy Short"],
                                                      data_quarter["Sortino-Portfolio Best_Strategy Short"],"1:0","0:1")
data_yearly["Short Dummy Y Bull/Bear"]=Second_Dependent_Variable(data_yearly["Sharpe-Portfolio Best_Strategy Short"],
                                                      data_yearly["Sortino-Portfolio Best_Strategy Short"],"1:0","0:1")

#### Removing empty observations

In [56]:
data_quarter=Remove_Character(data_quarter, data_quarter["Bull Sharpe Ratio Long"], ".")
data_yearly=Remove_Character(data_yearly, data_yearly["Bull Sharpe Ratio Long"], ".")

### Creating a one lag dependent variable

In [57]:
def Lag_Dependent_Variable (ticker, variable):
    Lag_Dependent_Variable = []
    for i in range (len(variable)):
        if i == len(variable)-1:
            Lag_Dependent_Variable.append(".")
        elif ticker[i]==ticker[i+1]:
            Lag_Dependent_Variable.append(variable[i+1])
        else:
            Lag_Dependent_Variable.append(".")
    return Lag_Dependent_Variable

In [58]:
data_quarter["Long Lag Dummy Y Stock/LETF"]= Lag_Dependent_Variable(data_quarter["Bull Ticker"], 
                                                                   data_quarter["Long Dummy Y Stock/LETF"])
data_quarter["Short Lag Dummy Y Stock/LETF"]=Lag_Dependent_Variable(data_quarter["Bull Ticker"], 
                                                                   data_quarter["Short Dummy Y Stock/LETF"])

In [59]:
data_yearly["Long Lag Dummy Y Stock/LETF"]=Lag_Dependent_Variable(data_yearly["Bull Ticker"], 
                                                                 data_yearly["Long Dummy Y Stock/LETF"])
data_yearly["Short Lag Dummy Y Stock/LETF"]=Lag_Dependent_Variable(data_yearly["Bull Ticker"], 
                                                                  data_yearly["Short Dummy Y Stock/LETF"])

In [60]:
data_quarter["Long Lag Dummy Y Bull/Bear"]= Lag_Dependent_Variable(data_quarter["Bull Ticker"], 
                                                                   data_quarter["Long Dummy Y Bull/Bear"])
data_quarter["Short Lag Dummy Y Bull/Bear"]=Lag_Dependent_Variable(data_quarter["Bull Ticker"], 
                                                                   data_quarter["Short Dummy Y Bull/Bear"])

In [61]:
data_yearly["Long Lag Dummy Y Bull/Bear"]=Lag_Dependent_Variable(data_yearly["Bull Ticker"], 
                                                                 data_yearly["Long Dummy Y Bull/Bear"])
data_yearly["Short Lag Dummy Y Bull/Bear"]=Lag_Dependent_Variable(data_yearly["Bull Ticker"], 
                                                                  data_yearly["Short Dummy Y Bull/Bear"])

In [62]:
del data_quarter["level_0"]
del data_yearly["level_0"]
del data_quarter["index"]
del data_yearly["index"]

### Saving the datasets

In [63]:
data_quarter.to_excel(r"D:\Elahe\MSF\Thesis\Datasets\Quarter1.xlsx")
data_yearly.to_excel(r"D:\Elahe\MSF\Thesis\Datasets\Year1.xlsx")

### Frequency of best strategies based on different measurments for quarter

In [64]:
data_quarter["CuRe-Portfolio Best_Strategy Long"].value_counts()

1:0    1105
0:1     479
Name: CuRe-Portfolio Best_Strategy Long, dtype: int64

In [65]:
data_quarter["Sharpe-Portfolio Best_Strategy Long"].value_counts()

1:0    1106
0:1     478
Name: Sharpe-Portfolio Best_Strategy Long, dtype: int64

In [66]:
data_quarter["Sortino-Portfolio Best_Strategy Long"].value_counts()

1:0    1107
0:1     477
Name: Sortino-Portfolio Best_Strategy Long, dtype: int64

In [67]:
data_quarter["EVaR-Portfolio Best_Strategy Long"].value_counts()

0:1    857
1:0    727
Name: EVaR-Portfolio Best_Strategy Long, dtype: int64

In [68]:
data_quarter["CuRe-Portfolio Best_Strategy Short"].value_counts()

0:1    1105
1:0     479
Name: CuRe-Portfolio Best_Strategy Short, dtype: int64

In [69]:
data_quarter["Sharpe-Portfolio Best_Strategy Short"].value_counts()

0:1    1107
1:0     477
Name: Sharpe-Portfolio Best_Strategy Short, dtype: int64

In [70]:
data_quarter["Sortino-Portfolio Best_Strategy Short"].value_counts()

0:1    1108
1:0     476
Name: Sortino-Portfolio Best_Strategy Short, dtype: int64

In [71]:
data_quarter["EVaR-Portfolio Best_Strategy Short"].value_counts()

1:0    864
0:1    720
Name: EVaR-Portfolio Best_Strategy Short, dtype: int64

### Frequency of best strategies based on different measurments for year

In [72]:
data_yearly["CuRe-Portfolio Best_Strategy Long"].value_counts()

1:0    294
0:1    102
Name: CuRe-Portfolio Best_Strategy Long, dtype: int64

In [73]:
data_yearly["Sharpe-Portfolio Best_Strategy Long"].value_counts()

1:0    293
0:1    103
Name: Sharpe-Portfolio Best_Strategy Long, dtype: int64

In [74]:
data_yearly["Sortino-Portfolio Best_Strategy Long"].value_counts()

1:0    294
0:1    102
Name: Sortino-Portfolio Best_Strategy Long, dtype: int64

In [75]:
data_yearly["EVaR-Portfolio Best_Strategy Long"].value_counts()

0:1    245
1:0    151
Name: EVaR-Portfolio Best_Strategy Long, dtype: int64

In [76]:
data_yearly["CuRe-Portfolio Best_Strategy Short"].value_counts()

0:1    294
1:0    102
Name: CuRe-Portfolio Best_Strategy Short, dtype: int64

In [77]:
data_yearly["Sharpe-Portfolio Best_Strategy Short"].value_counts()

0:1    294
1:0    102
Name: Sharpe-Portfolio Best_Strategy Short, dtype: int64

In [78]:
data_yearly["Sortino-Portfolio Best_Strategy Short"].value_counts()

0:1    294
1:0    102
Name: Sortino-Portfolio Best_Strategy Short, dtype: int64

In [79]:
data_yearly["EVaR-Portfolio Best_Strategy Short"].value_counts()

1:0    251
0:1    145
Name: EVaR-Portfolio Best_Strategy Short, dtype: int64